In [13]:
from importlib import reload
from burst_workflow import bursts
import pystac
import pandas as pd
reload(bursts)

<module 'burst_workflow.bursts' from '/Users/ffwilliams2/Repositories/burst_prototype/burst_workflow/bursts.py'>

In [2]:
cd ../example_data

/Users/ffwilliams2/Repositories/burst_prototype/example_data


In [3]:
slc = 'S1A_IW_SLC__1SSV_20151208T114941_20151208T115016_008950_00CD2C_58F9.zip'
df = pd.DataFrame()
burst_df = bursts.update_burst_dataframe(df, slc, swath=1, polarization='vv')
burst_df

,absoluteID,relativeID,swath,polarization,date,pass_direction,annotation,measurement,longitude,latitude,geometry,lines,samples,byte_offset,byte_length
0,S1_SLC_20151208T114941_VV_273859_IW1,273859,1,vv,20151208T114941,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-90.875643,0.812758,POLYGON ((-90.48049619041907 0.815441239601974...,1487,20473,143067,121773404
1,S1_SLC_20151208T114944_VV_273860_IW1,273860,1,vv,20151208T114944,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-90.910556,0.645977,"POLYGON ((-90.51541789826572 0.6484604633983, ...",1487,20473,121916471,121773404
2,S1_SLC_20151208T114946_VV_273861_IW1,273861,1,vv,20151208T114946,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-90.945455,0.479385,POLYGON ((-90.55030972169011 0.481730319191587...,1487,20473,243689875,121773404
3,S1_SLC_20151208T114949_VV_273862_IW1,273862,1,vv,20151208T114949,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-90.980407,0.312671,POLYGON ((-90.58519794402078 0.315126779162864...,1487,20473,365463279,121773404
4,S1_SLC_20151208T114952_VV_273863_IW1,273863,1,vv,20151208T114952,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-91.015464,0.145586,POLYGON ((-90.62018723387152 0.148152178048328...,1487,20473,487236683,121773404
5,S1_SLC_20151208T114955_VV_273864_IW1,273864,1,vv,20151208T114955,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-91.050521,-0.021372,POLYGON ((-90.65527819816886 -0.01919369315353...,1487,20473,609010087,121773404
6,S1_SLC_20151208T114957_VV_273865_IW1,273865,1,vv,20151208T114957,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-91.085567,-0.188140,POLYGON ((-90.6902623281414 -0.185914793366226...,1487,20473,730783491,121773404
7,S1_SLC_20151208T115000_VV_273866_IW1,273866,1,vv,20151208T115000,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-91.120615,-0.354781,POLYGON ((-90.72532284357422 -0.35288271633123...,1487,20473,852556895,121773404
8,S1_SLC_20151208T115003_VV_273867_IW1,273867,1,vv,20151208T115003,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-91.155704,-0.521482,POLYGON ((-90.76030323792682 -0.51935046751119...,1487,20473,974330299,121773404
9,S1_SLC_20151208T115006_VV_273868_IW1,273868,1,vv,20151208T115006,Descending,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,S1A_IW_SLC__1SSV_20151208T114941_20151208T1150...,-91.190848,-0.688305,POLYGON ((-90.79543930537002 -0.68643808695673...,1487,20473,1096103703,121773404


In [4]:
stac = bursts.generate_stac_catalog(burst_df)

In [5]:
stac_file = bursts.save_stac_catalog_locally(stac)

In [6]:
stac.describe()

* <Catalog id=burst-catalog>
  * <Item id=S1_SLC_20151208T114941_VV_273859_IW1>
  * <Item id=S1_SLC_20151208T114944_VV_273860_IW1>
  * <Item id=S1_SLC_20151208T114946_VV_273861_IW1>
  * <Item id=S1_SLC_20151208T114949_VV_273862_IW1>
  * <Item id=S1_SLC_20151208T114952_VV_273863_IW1>
  * <Item id=S1_SLC_20151208T114955_VV_273864_IW1>
  * <Item id=S1_SLC_20151208T114957_VV_273865_IW1>
  * <Item id=S1_SLC_20151208T115000_VV_273866_IW1>
  * <Item id=S1_SLC_20151208T115003_VV_273867_IW1>
  * <Item id=S1_SLC_20151208T115006_VV_273868_IW1>
  * <Item id=S1_SLC_20151208T115008_VV_273869_IW1>
  * <Item id=S1_SLC_20151208T115011_VV_273870_IW1>


In [7]:
# bursts.initiate_stac_catalog_server(5555,stac_file.parent)

In [8]:
stac = pystac.Catalog.from_file(stac_file.__str__())

In [9]:
stac.describe()

* <Catalog id=burst-catalog>
  * <Item id=S1_SLC_20151208T114941_VV_273859_IW1>
  * <Item id=S1_SLC_20151208T114944_VV_273860_IW1>
  * <Item id=S1_SLC_20151208T114946_VV_273861_IW1>
  * <Item id=S1_SLC_20151208T114949_VV_273862_IW1>
  * <Item id=S1_SLC_20151208T114952_VV_273863_IW1>
  * <Item id=S1_SLC_20151208T114955_VV_273864_IW1>
  * <Item id=S1_SLC_20151208T114957_VV_273865_IW1>
  * <Item id=S1_SLC_20151208T115000_VV_273866_IW1>
  * <Item id=S1_SLC_20151208T115003_VV_273867_IW1>
  * <Item id=S1_SLC_20151208T115006_VV_273868_IW1>
  * <Item id=S1_SLC_20151208T115008_VV_273869_IW1>
  * <Item id=S1_SLC_20151208T115011_VV_273870_IW1>


In [10]:
item = stac.get_item('S1_SLC_20151208T114941_VV_273859_IW1', recursive=True)

In [11]:
item.properties

{'lines': 1487,
 'samples': 20473,
 'byte_offset': 143067,
 'byte_length': 121773404,
 'datetime': '2015-12-08T11:49:41Z'}

In [14]:
burst_arr = bursts.read_local_burst(item)

In [15]:
burst_arr.shape

(1487, 20473)

In [ ]:
# For Later:
eo_item.ext.enable(pystac.Extensions.EO)
eo_item.ext.eo.apply(bands=wv3_bands)